# Запуск имитационной модели системы массового обслуживания с приоритетами

Имитационная модель (ИМ) СМО поддерживает различные типы распределений входного потока и обслуживания, произвольное число каналов. Возможно задание максимальной длины очереди. По умолчанию - бесконечная очередь. В нотации Кендалла: GI/G/n/r и GI/G/n.

Для задания приоритета необходимо передать в ИМ тип приоритета *prty_type*

| prty_type |                                     Тип приоритета                                     |
|:---------:|:----------------------------------------------------------------------:|
|    No     |                                 без приоритетов, FIFO                                  |
|    PR     |                 preemptive resume, с дообслуживанием прерванной заявки                 |
|    RS     | preemptive repeat with resampling, обслуживание заново с новой случайной длительностью |
|    RW     |   preemptive repeat without resampling, обслуживание заново с прежней длительностью    |
|    NP     |                        non preemptive, относительный приоритет                         |

Для запуска ИМ требуется 4 шага:
- создание экземпляра ИМ
- задание входного потока
- задание распределения обслуживания
- запуск модели


#### Задаем число каналов, число классов, интенсивности вх потоков для каждого класса заявок

In [11]:
import rand_destribution as rd

n = 5  # число каналов обслуживания
k = 3  # число классов
l = [0.2, 0.3, 0.4]
lsum = sum(l)

#### Задаем начальные моменты распределений обслуживания.

Для дальнейшей сверки с численными методами необходимо задать число начальных моментов на 1 больше, чем требуемое число начальных моментов распределения времени пребывания заявок в системе. Т.е. для получения v1, v2, v3 требуется задание 4-х моментов для каждого класса заявок.
По заданным моментам можем получить параметры аппроксимирующего распределения для задания ИМ. В данном случае распределение времени обслуживания задано Гамма-распределением. Для нахождения двух его параметров требуется два начальных момента.

In [12]:
# Первый и второй начальные моменты времени обслуживания для каждого из 3-х классов
b1 = [0.45 * n, 0.9 * n, 1.35 * n]
b2 = [0] * k
coev = 0.577

for i in range(k):
    b2[i] = (b1[i] ** 2) * (1 + coev ** 2)

# Среднее время обслуживания и коэффициент загрузка
b_sr = sum(b1) / k
ro = lsum * b_sr / n

print(f'ro = {ro:0.3f}')

# Параметры Гамма-распределения
params = []
for i in range(k):
    params.append(rd.Gamma.get_mu_alpha([b1[i], b2[i]]))

print('\nПараметры Гамма-распределения для каждого класса:\n', params)

# По полученным параметрам можем получить 4 момента
b = []
for j in range(k):
    b.append(rd.Gamma.calc_theory_moments(params[j][0], params[j][1], 4))

print('\nНачальные моменты\n', b)

ro = 0.810

Параметры Гамма-распределения для каждого класса:
 [(1.3349526308745838, 3.0036434194678137), (0.6674763154372919, 3.0036434194678137), (0.44498421029152807, 3.0036434194678145)]

Начальные моменты
 [[2.25, 6.747953062500001, 25.292546083777786, 113.7474276956009], [4.5, 26.991812250000002, 202.3403686702223, 1819.9588431296145], [6.75, 60.73157756250001, 682.8987442620003, 9213.541643343671]]


#### Импорт класса PriorityQueueSimulator и создание экземпляра ИМ.
При создании укажем число каналов *n* и число классов *k*. Также укажем тип приоритета - *PR* (абсолютный приоритет).

In [13]:
from most_queue.sim.priority_queue_sim import PriorityQueueSimulator

qs = PriorityQueueSimulator(n, k, "PR")

#### Зададим параметры вх потоков и обслуживания

In [14]:
sources = []
servers_params = []
for j in range(k):
    sources.append({'type': 'M', 'params': l[j]})
    servers_params.append({'type': 'Gamma', 'params': params[j]})

qs.set_sources(sources)
qs.set_servers(servers_params)

#### Для запуска ИМ необходимо вызвать метод *run* и передать число заявок

In [15]:
num_of_jobs = 300000
qs.run(num_of_jobs)
v_sim = qs.v
print(v_sim)

Start simulation



100%|██████████| 300000/300000 [00:01<00:00, 225724.28it/s]


Simulation is finished

[[2.2555726774927884, 6.765895932885055, 25.316381696950828], [4.89295833562666, 32.66972034395344, 277.41872504558796], [17.648183522210264, 606.2721315766971, 37662.597415239026]]


#### Численный расчет методом инвариантов отношений

In [16]:
from most_queue.theory import priority_calc
v_teor = priority_calc.get_v_prty_invar(l, b, n, 'PR')

#### Вывод результатов

In [18]:
from most_queue.utils.tables import times_print_with_classes
print("\nСравнение данных ИМ и результатов расчета методом инвариантов отношения (Р) \n"
          "для времени пребывания в многоканальной СМО с приоритетами")
print(f"Число каналов: {n}\nЧисло классов: {k}\nКоэффициент загрузки: {ro:<1.2f}\n")
print(f"Коэффициент вариации времени обслуживания {coev}\n")
print("Абсолютный приоритет")

times_print_with_classes(v_sim, v_teor, is_w=False)


Сравнение данных ИМ и результатов расчета методом инвариантов отношения (Р) 
для времени пребывания в многоканальной СМО с приоритетами
Число каналов: 5
Число классов: 3
Коэффициент загрузки: 0.81

Коэффициент вариации времени обслуживания 0.577

Абсолютный приоритет
   Начальные моменты времени пребывания заявок в системе    
------------------------------------------------------------
           |           Номер начального момента            |
   № кл    | --------------------------------------------- |
           |       1       |       2       |       3       |
------------------------------------------------------------
     | ИМ  |     2.26      |     6.77      |     25.3      |
  1  |------------------------------------------------------
     |  Р  |     2.27      |     6.85      |     25.8      |
------------------------------------------------------------
     | ИМ  |     4.89      |     32.7      |      277      |
  2  |------------------------------------------------------